In [22]:
import numpy as np
import gmsh


gmsh.initialize()
gmsh.model.add('PROBLEMA1')


# Generado de la figura
L1 = 20
L2 = 10
Lc = 20 #caso del ejercicio con 4 triangulos
#Lc = 1

p1 = gmsh.model.geo.addPoint(0, 0, 0, Lc)
p2 = gmsh.model.geo.addPoint(0, L2, 0, Lc) 
p3 = gmsh.model.geo.addPoint(L1, L2, 0, Lc) 
p4 = gmsh.model.geo.addPoint(L1, 0, 0, Lc)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

S1 = gmsh.model.geo.addPlaneSurface([C1])




#Generado de las condiciones de contorno

Empotrado = gmsh.model.addPhysicalGroup(1, [l1]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,Empotrado,'Empotrado')

Traccionado = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado') #esta funcion le pone nombre a todos los nodos aca

Superficie = gmsh.model.addPhysicalGroup(2,[S1]) #2 es para superficies!
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')


gmsh.model.geo.synchronize() #sincroniza lo que creamos con el modelo que creamos arriba
gmsh.model.mesh.generate(2)

#gmsh.fltk.run()

In [23]:
NodeInfo = gmsh.model.mesh.get_nodes()
NodeInfo[1].shape
NumeroNodos = NodeInfo[0].shape[0]
MN = NodeInfo[1].reshape(NumeroNodos , 3)


ElementInfo = gmsh.model.mesh.get_elements()
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])


In [24]:
#Empotrados
NodosEmpotrados = gmsh.model.mesh.get_nodes_for_physical_group(1,Empotrado)
A = NodosEmpotrados[0].reshape(-1,1)
B = np.zeros((len(A),2))
MNE = np.concatenate((A,B) , axis = 1)

#Traccionados
NodosTraccionados = gmsh.model.mesh.get_nodes_for_physical_group(1, Traccionado)
C = NodosTraccionados[0].reshape(-1,1)
D = np.zeros((len(C),2))
MNT = np.concatenate((C,D) , axis = 1)
MNT[:,1] = 5000



#resto?

In [25]:
Nodoslibres = []

Nodos = gmsh.model.mesh.get_nodes_for_physical_group(2, Superficie)
for i in Nodos[0]:
    if i in NodosTraccionados[0]:
        l = 0
    elif i in NodosEmpotrados[0]:
        l = 0
    else:
        Nodoslibres.append(i)
        
E = np.array(Nodoslibres).reshape(-1,1)
D = np.zeros((len(E),2))
Interior = np.concatenate((E,D) , axis = 1)
MNT = np.concatenate((Interior,MNT) , axis = 0)

gmsh.finalize()

In [5]:
#MNT[np.argsort(MNT[:,0])]


In [6]:
print(MN)
print()
print(MC)
print()
print(MNE)
print()
print(MNT)

[[ 0.  0.  0.]
 [ 0. 10.  0.]
 [20. 10.  0.]
 [20.  0.  0.]
 [10.  5.  0.]]

[[3 4 5]
 [1 2 5]
 [4 1 5]
 [2 3 5]]

[[1. 0. 0.]
 [2. 0. 0.]]

[[5.e+00 0.e+00 0.e+00]
 [3.e+00 5.e+03 0.e+00]
 [4.e+00 5.e+03 0.e+00]]


In [26]:
espesor = 1
E = 30E6
v = 0.3

In [45]:
D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []
Areas = []

In [46]:
for i in range (len(MC)):
    
    I,M,J = MC[i]    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)
    
    x = np.array([MN[i,0],MN[m,0],MN[j,0]])
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])
    #Bm = y[0] - y[2] ;   Bi = y[2] - y[1] ;    Bj = y[1] - y[0]
    #Gi = x[1] - x[2] ;   Gm = x[2] - x[0] ;    Gj = x[0] - x[1]  
    Bi = y[1] - y[2] ;   Bm = y[2] - y[0] ;    Bj = y[0] - y[1]
    Gi = x[2] - x[1] ;   Gm = x[0] - x[2] ;    Gj = x[1] - x[0]      
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    Area = np.linalg.det(Matriz_coordenadas)/2

   # Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bj,0,Bm,0],[0,Gi,0,Gj,0,Gm],[Gi,Bi,Gj,Bj,Gm,Bm]])
    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bm,0,Bj,0],[0,Gi,0,Gm,0,Gj],[Gi,Bi,Gm,Bm,Gj,Bj]])
    Matrices_Bloc.append(Matriz1)
    Matriz_local = espesor * abs(Area) * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,m*2,m*2+1,j*2,j*2+1]) #aca pasa algo raro
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    
print(Matriz_global*(0.91/375000))

[[ 48.   26.   -4.    1.    0.    0.    4.   -1.  -48.  -26. ]
 [ 26.   87.   -1.  -36.5   0.    0.    1.   36.5 -26.  -87. ]
 [ -4.   -1.   48.  -26.    4.    1.    0.    0.  -48.   26. ]
 [  1.  -36.5 -26.   87.   -1.   36.5   0.    0.   26.  -87. ]
 [  0.    0.    4.   -1.   48.   26.   -4.    1.  -48.  -26. ]
 [  0.    0.    1.   36.5  26.   87.   -1.  -36.5 -26.  -87. ]
 [  4.    1.    0.    0.   -4.   -1.   48.  -26.  -48.   26. ]
 [ -1.   36.5   0.    0.    1.  -36.5 -26.   87.   26.  -87. ]
 [-48.  -26.  -48.   26.  -48.  -26.  -48.   26.  192.    0. ]
 [-26.  -87.   26.  -87.  -26.  -87.   26.  -87.    0.  348. ]]


In [47]:
Desplazamientos_f = np.zeros(len(MN)*2)
Fuerza = []
Nodos = []


for i in MNE:
    Nodo,valorx,valory = i 
    nodo = int(Nodo-1)
    Desplazamientos_f[2*nodo] = valorx
    Desplazamientos_f[2*nodo+1] = valory


for i in MNT:
    Nodo,valorx,valory = i
    nodo = int(Nodo-1)
    Fuerza.append(valorx)
    Fuerza.append(valory)
    Nodos.append(2*nodo)
    Nodos.append(2*nodo+1) 


Fuerza = np.array(Fuerza).reshape(-1,1)

Matriz_res = Matriz_global[np.ix_(Nodos,Nodos)]

Desplazamientos_parcial = np.linalg.solve(Matriz_res,Fuerza)


o = 0
for i in Nodos:
    Desplazamientos_f[i] = Desplazamientos_parcial [o]
    o += 1
  

Fuerzas_ = np.dot(Matriz_global,Desplazamientos_f)
print(Fuerzas_.round(4))
print()
print(Desplazamientos_f*1e6)

[-5000.     -2002.1356 -5000.      2002.1356  5000.        -0.
  5000.        -0.         0.         0.    ]

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.46645311e+02 -6.66310731e+01  6.46645311e+02  6.66310731e+01
  3.05276740e+02 -2.45232180e-13]


In [48]:
#Esfuerzos

Tensiones = []

for p in range(len(MC)):
    
    I,M,J = MC[p]    
    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)

    
    nodos = [2*i,2*i+1,2*m,2*m+1,2*j,2*j+1]
    
    
    A = np.dot(Matrices_Bloc[p],Desplazamientos_f[nodos])
    
    B = np.dot(D,A)

   
    Tensiones.append(B)

for i in range (len(Tensiones)):
    print(Tensiones[i])
    print()

[ 993.59316604 -101.70848905    0.        ]

[ 1.00640683e+03  3.01922050e+02 -2.82960208e-13]

[1000.          100.10678057   -3.20341698]

[1000.          100.10678057    3.20341698]

